# Trabalho Prático 3 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Esquemas pós-quânticos de assinaturas digitais* - qTESLA

Nota: Baseado no documento qTESLA (https://qtesla.org/wp-content/uploads/2020/04/qTESLA_round2_14.04.2020.pdf)

### Parâmetros utilizados

- `n`: dimensão
- `k`: número de amostras
- `q`: módulo
- `h`: número de entradas diferentes de 0 nos elementos retornados pela função `Enc`
- `K`: comprimento do output da função `H` e do input da `GenA` e `Enc`.
- `Le`: limite de `checkE`
- `Ls`: limite de `checkS`
- `B`: determina o intervalo de aleatoriedade durante a assinatura.
- `d`: número de bits aleatórios

In [1]:
import numpy as np
import hashlib

class qTESLA(object):
    
    def __init__(self, K, k, h, Le, Ls, B, d, n, q, timeout=None):
        
        self.K = K
        self.k = k
        self.h = h
        self.Le = Le
        self.Ls = Ls
        self.B = B
        self.d = d
        self.n = n
        self.q = q
        
        Zx.<x>  = ZZ[]
        self.Zx = Zx
        Gq.<z>  = GF(q)[]
        
        R.<x> = Zx.quotient(x^n+1)  # R
        self.R = R
        Rq.<z> = Gq.quotient(z^n+1) # R/q
        self.Rq = Rq
    
    
    def Hash(self, s):
        
        h = hashlib.sha256()
        h.update(s)
        return h.digest()
    
    
    def binary(self, size):
        
        return list(np.random.choice([0,1],size))  
    
    
    def GenA(self):
        
        return self.Rq.random_element()
    
    
    def _center_lift(self, x):
        
        return lift(x + self.q//2) - self.q//2
    
    
    def _round(self, w):
        
        return self.Zx(list(map(lambda x: self._center_lift(x), w.list())))
    
    
    def checkS(self, s):
        
        sum = 0
        ls = list(s)
        ls.sort(reverse=True)
        for i in range(0, self.h):
            sum += ls[i]
        if sum > self.Ls:
            return 1
        return 0
    
    
    def checkE(self, s):
        
        sum_ = 0
        ls = s.list()
        ls.sort(reverse=True)
        for i in range(0, self.h):
            sum_ += int(ls[i])
        if sum_ > self.Le:
            return 1
        return 0
    
    
    def H(self, v, hash_m):
        
        w = [0] * self.n
        for i in range(self.n):
            val = v[i] % 2^self.d
            if val > 2^(self.d-1):
                val = val - 2^self.d
            w[i] =  (v[i] - val)/2^self.d
        return self.Hash(str(w).encode()+hash_m)
    
    
    def Enc(self, c):
        
        D = 0
        cnt = 0
        r = hashlib.shake_128(c + bytes(D)).digest(int(168))
        i = 0
        pos_list = [None] * self.h
        sign_list = [None] * self.h
        c_ = [0] * 512
        while i < self.h:
            if cnt > 168-3:
                D += 1
                cnt = 0
                r = hashlib.shake_128(c + bytes(D)).digest(int(168))
            pos = (r[cnt] * 2^8 + r[cnt+1]) % self.n
            if c_[pos]==0:
                if r[cnt+2] % 2 == 1:
                    c_[pos] =  -1
                else:
                    c_[pos] = 1
                pos_list[i] = pos
                sign_list[i] = c_[pos]
                i += 1
            cnt += 3
        return pos_list, sign_list
    
    
    def genPol(self, pos_list, sign_list):
        
        r = [0]*512
        j = 0
        for i in pos_list:
            r[i] = sign_list[j]
            j+=1
        return self.Rq(r)
    
    
    def keyGen(self):
        
        a = self.GenA()
        s = None
        while (True):
            s = self.Rq.random_element(distribution="gaussian")
            if self.checkS(s) == 0:
                break
        
        e = None
        while (True):
            e = self.Rq([abs(ZZ.random_element(x=self.Le // 60, distribution="gaussian")) for _ in range(512)])
            if self.checkE(e) == 0:
                break
        
        t = a*s + e
        sk = (s, e, a)
        pk = (a, t)
        return sk, pk
    
    
    def sign(self, m, sk):
        
        s, e, a = sk
        y = self.Rq.random_element(x=-self.B, y=self.B+1, distribution="uniform")
        v = self._round(a*y)
        c1 = self.H(v, self.Hash(str(m).encode()))
        c2 = self.Enc(c1)
        z = y + s * self.genPol(c2[0],c2[1])
        return (z, c1)
    
    
    def verify(self, m, s, pk):
        
        z, c1 = s
        c2 = self.Enc(c1)
        a, t = pk
        w = self._round(a*z - t*self.genPol(c2[0],c2[1]))
        if c1 != self.H(w, self.Hash(str(m).encode())):
            return False
        return True
        

#### Testagem da classe definida acima:

In [4]:
# Parametros iniciais
K = 256
k = 1
h = 30
Le = 1586
Ls = 1586
B = 2^20-1
d = 21
n = 512
q = 4205569

qtesla = qTESLA(K, k, h, Le, Ls, B, d, n, q)

mess = qtesla.binary(n)

sk, pk = qtesla.keyGen()

s = qtesla.sign(mess,sk)

res = qtesla.verify(mess, s, pk)
if res==True:
    print("A assinatura digital é válida!!")
else:
    print("A assinatura digital não é válida!!")

A assinatura digital é válida!!
